Imports

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import os
from PIL import Image
import numpy as np
import pandas as pd

In [3]:
def maximum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.max(all_weights, axis=0))
    return aggregated_weights, "MaximumWeight"

def minimum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.min(all_weights, axis=0))
    return aggregated_weights, "MinimumWeight"

def median_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.median(all_weights, axis=0))
    return aggregated_weights, "MedianWeight"

def trimmed_mean_aggregation(models_states, trim_fraction=0.2):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        sorted_weights = np.sort(all_weights, axis=0)
        trim_size = int(trim_fraction * len(sorted_weights))
        trimmed_weights = sorted_weights[trim_size:-trim_size]
        aggregated_weights[param_name] = torch.tensor(np.mean(trimmed_weights, axis=0))
    return aggregated_weights, "TrimmedMean"

def geometric_mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.exp(np.mean(np.log(all_weights), axis=0)))
    return aggregated_weights, "GeometricMean"

def harmonic_mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(len(all_weights) / np.sum(1.0 / all_weights, axis=0))
    return aggregated_weights, "HarmonicMean"

def weighted_average_aggregation(models_states, model_weights):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [model_state[param_name] for model_state in models_states]
        weights_array = np.array(model_weights)
        normalized_weights = weights_array / np.sum(weights_array)
        aggregated_weights[param_name] = torch.tensor(np.sum(normalized_weights * all_weights, axis=0))
    return aggregated_weights, "WeightedAverage"

In [4]:
# Define CarDataSet Class
class CarDataSet():

    # Define The Initialization
    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        self.cars = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.resize = transforms.Resize((150,150))  # Resize images to a uniform size

    # Define The Length Function
    def __len__(self):
        return len(self.cars)

    # Define The Get Item Function
    def __getitem__(self, idx):

        # Pull The Image And Check Settings
        img_name = os.path.join(self.root_dir, self.cars.iloc[idx, 0])

        image = Image.open(img_name)

        if image.mode != 'RGB':
          image = image.convert('RGB')

        # Pull The Label, -1 To Normalize To 0
        label = (self.cars.iloc[idx, 5]) - 1

        if self.transform:
            image = self.transform(image)

        # Define The Dictionary
        sample = {'image': image, 'cars': label}

        # Return
        return sample
    
# Test Accuracy
def test_accuracy(model, test_loader_internal, passed_device, loss_fn):

    # Set Parameters
    model.to(passed_device)
    correct = 0
    total = 0
    run = 0
    val_loss = 0

    # Run Tests
    with torch.no_grad():
        for test_data_internal in test_loader_internal:
            images_test_acc, labels_test_acc = test_data_internal['image'].cuda(), test_data_internal['cars'].cuda()
            outputs_test_acc = model(images_test_acc)
            _, predicted_test_acc = torch.max(outputs_test_acc.data, 1)
            val_loss += (loss_fn(outputs_test_acc, labels_test_acc)).item()
            total += labels_test_acc.size(0)
            correct += (predicted_test_acc == labels_test_acc).sum().item()
            run += 1

    # Return
    return (100 * correct / total), val_loss/run

In [5]:
# Define Transform
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    
# Load The Data
test_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/val_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/val', transform=transform)

train_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/train_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/train', transform=transform)

In [6]:
InternalSave = pd.read_csv("../SynchronizationProject/SaveData/Test/InfoSave.csv")

In [7]:
InternalSave

,Run,BatchSize,LearningRate,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,GPUNumber,PreTrain,SavePathInput,SavePathOutput
0,0,32,0.0001,29.766119,2.530721,"{'loss': [3.4930564347435444, 3.41162627351050...",16.673579,3.380552,"{'loss': [3.5342165074850382, 3.59506788692976...",1,1,False,NaN,../SynchronizationProject/SaveData/Weights/0.pth
1,1,32,0.0001,22.628991,2.912521,"{'loss': [3.5299864712883444, 3.43312999500947...",9.373704,3.575293,"{'loss': [3.5124278695959794, 3.49547933904748...",3,1,False,NaN,../SynchronizationProject/SaveData/Weights/1T0...
2,1,32,0.0001,19.352325,3.014417,"{'loss': [3.5531147339764764, 3.45249549641328...",12.982165,3.580856,"{'loss': [3.520307826368432, 3.501597238214392...",3,2,False,NaN,../SynchronizationProject/SaveData/Weights/1T1...
3,1,32,0.0001,14.903829,3.129956,"{'loss': [3.557936040092917, 3.476790214987362...",12.567399,3.436109,"{'loss': [3.5215154917616593, 3.47128234411540...",3,3,False,NaN,../SynchronizationProject/SaveData/Weights/1T2...
4,2,64,0.0001,25.569260,2.713737,"{'loss': [3.4837466087192297, 3.39986238069832...",15.429282,3.255348,"{'loss': [3.4475874649850944, 3.41609198168704...",1,1,False,NaN,../SynchronizationProject/SaveData/Weights/2.pth
5,3,64,0.0001,23.005913,3.016223,"{'loss': [3.600028001345121, 3.422101405950693...",8.336790,3.701210,"{'loss': [3.5449120433706987, 3.51157576159427...",5,1,False,NaN,../SynchronizationProject/SaveData/Weights/3T0...
6,3,64,0.0001,25.058370,2.930626,"{'loss': [3.5564707059126635, 3.39519774913787...",10.991290,3.682562,"{'loss': [3.5440946942881535, 3.49703889144094...",5,2,False,NaN,../SynchronizationProject/SaveData/Weights/3T1...
7,3,64,0.0001,20.478625,2.953434,"{'loss': [3.598517115299518, 3.457311208431537...",7.341352,3.960901,"{'loss': [3.4908311680743567, 3.49983194627259...",5,3,False,NaN,../SynchronizationProject/SaveData/Weights/3T2...
8,3,64,0.0001,25.610257,2.911658,"{'loss': [3.601519465446472, 3.431241613167983...",8.212360,4.160569,"{'loss': [3.5688967642031217, 3.50579348363374...",5,4,False,NaN,../SynchronizationProject/SaveData/Weights/3T3...
9,3,64,0.0001,22.036984,3.032949,"{'loss': [3.5894748889482937, 3.40311352106241...",9.290751,3.795039,"{'loss': [3.5851886084205224, 3.53122147760893...",5,5,False,NaN,../SynchronizationProject/SaveData/Weights/3T4...


In [9]:
total_runs = InternalSave["Run"].iloc[-1]
first_occurrence_indices = InternalSave.groupby('Run').first()

In [15]:
specific_run_rows = InternalSave[InternalSave['Run'] == 3]

In [16]:
specific_run_rows

,Run,BatchSize,LearningRate,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,GPUNumber,PreTrain,SavePathInput,SavePathOutput
5,3,64,0.0001,23.005913,3.016223,"{'loss': [3.600028001345121, 3.422101405950693...",8.336790,3.701210,"{'loss': [3.5449120433706987, 3.51157576159427...",5,1,False,NaN,../SynchronizationProject/SaveData/Weights/3T0...
6,3,64,0.0001,25.058370,2.930626,"{'loss': [3.5564707059126635, 3.39519774913787...",10.991290,3.682562,"{'loss': [3.5440946942881535, 3.49703889144094...",5,2,False,NaN,../SynchronizationProject/SaveData/Weights/3T1...
7,3,64,0.0001,20.478625,2.953434,"{'loss': [3.598517115299518, 3.457311208431537...",7.341352,3.960901,"{'loss': [3.4908311680743567, 3.49983194627259...",5,3,False,NaN,../SynchronizationProject/SaveData/Weights/3T2...
8,3,64,0.0001,25.610257,2.911658,"{'loss': [3.601519465446472, 3.431241613167983...",8.212360,4.160569,"{'loss': [3.5688967642031217, 3.50579348363374...",5,4,False,NaN,../SynchronizationProject/SaveData/Weights/3T3...
9,3,64,0.0001,22.036984,3.032949,"{'loss': [3.5894748889482937, 3.40311352106241...",9.290751,3.795039,"{'loss': [3.5851886084205224, 3.53122147760893...",5,5,False,NaN,../SynchronizationProject/SaveData/Weights/3T4...


In [27]:
outputWeights = pd.DataFrame(columns = ["Run", "BatchSize", "LearningRate", "Epochs", "TrainAccuracy", "TrainLoss", "ValAccuracy", "ValLoss", "GPUCount", "MergeType", "PreTrain", "TrainTime", "SavePathOutput"]);

for run_number in range(0, total_runs + 1):
    specific_run_rows = InternalSave[InternalSave['Run'] == run_number]
    learning_rate = specific_run_rows["LearningRate"].iloc[0]
    batch_size = specific_run_rows["BatchSize"].iloc[0]
    epochs = specific_run_rows["Epochs"].iloc[0]
    train_time = specific_run_rows["TrainTime"].iloc[0]
    pretrain = specific_run_rows["PreTrain"].iloc[0]
    
    
    train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
    if len(specific_run_rows) == 1:
        
        # Model Definition and Final Layer Edit
        net = models.resnet50()
        net.fc = nn.Linear(net.fc.in_features, 49)
        
        net.load_state_dict(torch.load(specific_run_rows["SavePathOutput"]))
        
         # Load Model Onto GPU
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        net.to(device)
        
        # Loss Function and Optimizer
        loss_function = nn.CrossEntropyLoss()
        
        trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
        valacc, valloss = test_accuracy(net, test_loader, device, loss_function)
        
        PATH = '../SynchronizationProject/SaveData/ProcessedWeights/' + str(run_number) + '.pth'
        torch.save(net.state_dict(), PATH)
        
        new_data_row = {
            "Run": run_number,
            "BatchSize": batch_size,
            "LearningRate": learning_rate, 
            "Epochs": epochs,
            "TrainAccuracy": trainacc,
            "TrainLoss": trainloss,
            "ValAccuracy": valacc,
            "ValLoss": valloss,
            "GPUCount": 1,
            "MergeType": "NONE",
            "PreTrain": pretrain, 
            "TrainTime": train_time,
            "SavePathOutput": PATH
        }
    else:
        weightsList = []
        for pathindex in range(0,len(specific_run_rows)):
            weightsList.append(torch.load(list(specific_run_rows["SavePathOutput"])[pathindex]))
        
        aggregatedData = []
        aggregatedData.append(maximum_weight_aggregation(weightsList))
        aggregatedData.append(minimum_weight_aggregation(weightsList))
        aggregatedData.append(median_aggregation(weightsList))
        aggregatedData.append(trimmed_mean_aggregation(weightsList, trim_fraction=0.2))
        aggregatedData.append(geometric_mean_aggregation(weightsList))
        aggregatedData.append(harmonic_mean_aggregation(weightsList))
        #aggregatedData.append(weighted_average_aggregation(weightsList, model_weights))  Decide on this later    
        
        for item in aggregatedData:
            # Model Definition and Final Layer Edit
            net = models.resnet50()
            net.fc = nn.Linear(net.fc.in_features, 49)
            
            net.load_state_dict(item[0])
            
             # Load Model Onto GPU
            device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            net.to(device)
            
            # Loss Function and Optimizer
            loss_function = nn.CrossEntropyLoss()
            
            trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
            valacc, valloss = test_accuracy(net, test_loader, device, loss_function)
            
            PATH = '../SynchronizationProject/SaveData/ProcessedWeights/' + str(run_number) + str(item[1]) + '.pth'
            torch.save(net.state_dict(), PATH)
            
            new_data_row = {
                "Run": run_number, 
                "BatchSize": batch_size, 
                "LearningRate": learning_rate, 
                "Epochs": epochs, 
                "TrainAccuracy": trainacc, 
                "TrainLoss": trainloss, 
                "ValAccuracy": valacc, 
                "ValLoss": valloss, 
                "GPUCount": len(specific_run_rows), 
                "MergeType": item[1], 
                "PreTrain": pretrain, 
                "TrainTime": train_time,
                "SavePathOutput": PATH
            }

KeyError: 'Epochs'